In [8]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [9]:
# url to scrape
url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=laptop&_sacat=175672&_ipg=60"

In [10]:
def get_content_per_page(url, pages):
    html_contents = []
    for i in pages:
        
        webpage_response = requests.get(url+"&_pgn={}".format(i))
        soup = bs(webpage_response.content, "html.parser")
        html_contents.append(soup)
      
    return html_contents
        

In [11]:
contents = get_content_per_page(url, [1, 2, 3, 4])

In [12]:
def get_data_per_page(contents=contents):
    ebay_data = {'title':[], 'price':[], 'location':[]}
    for content in contents:
        for i in range(1, 61):
            ul_entity = content.find("ul",
                                     {"class":"srp-results srp-list clearfix"}).find_all('li',
                                                                                             {'data-view':'mi:1686|iid:{}'.format(i)})
            for li in ul_entity:
                title = li.find("div", {"class":"s-item__info clearfix"}).find("h3",{"class":"s-item__title"}).text
                price = li.find("div", {"class":"s-item__detail s-item__detail--primary"}).text
                location = li.find("span", {"class":"s-item__location s-item__itemLocation"}).text
                ebay_data['title'].append(title.strip())
                ebay_data['price'].append(price.strip())
                ebay_data['location'].append(location.strip())
        
        
    return ebay_data
            
        
    

In [13]:
data = get_data_per_page()

In [14]:
df = pd.DataFrame(data)

In [15]:
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 999)

In [16]:
df.head()

,title,price,location
0,CHUWI 11.6in Intel N4020 Laptop Computer Windows 10 4GB 128GB SSD Windows 10,$198.99,from United States
1,Dell Laptop Latitude Intel i5 16GB RAM 1TB HDD WIFI HDMI Windows 10 Pro PC,$209.95 to $429.95,from United States
2,"HP Chromebook 14-SMB 14"" Laptop Intel Celeron 2GB RAM 16GB eMMC",$47.98,from United States
3,"HP ProBook 4430s 14"" Laptop Computer PC Celeron 8GB Ram 500GB HD WiFi Windows 10",$159.99,from United States
4,"Lenovo Legion 5 17.3"" 144Hz Gaming Laptop R5-5600H 16GB RAM 1TB SSD RTX 3060",$999.99,from United States


In [220]:
df.shape

(240, 3)

In [225]:
# convert to csv
df.to_csv('ebay_product_info.csv', index=False)